In [5]:
d = {'AAPL': 'USD', 'RR.L': 'GBP', 'MSFT': 'USD', 'NVDA': 'USD', 'LDO.MI': 'EUR', '4816.T': 'JPY'}
v = list(set(d.values()))
print(v)
print(type(v))

['USD', 'GBP', 'JPY', 'EUR']
<class 'list'>


# ----------------------------------
# Import modules
# ----------------------------------

In [4]:
import os
import requests                      # HTTP client for API calls
import pandas as pd                  # Tabular data handling
from datetime import datetime        # Datetime handling
from typing import Iterable, Optional, Dict, Union
import matplotlib.pyplot as plt
%pip install yfinance
import yfinance as yf
from src.debug_print import debug_print

Note: you may need to restart the kernel to use updated packages.


# ----------------------------------
# Import functions
# ----------------------------------

In [5]:

from src.debug_print import debug_print
from src.exchange_rates import get_exchange_rates, get_crypto_prices, get_fx_and_crypto, get_share_prices, convert_prices_to_base_currency, plot_fx_timeseries

# ----------------------------------
# Set up variables
# ----------------------------------

In [6]:

base_currency = "GBP"
target_currencies = ["USD", "EUR", "JPY"]
cryptos = ["BTC", "ETH"]
shares = ['AAPL', 'RR.L', 'MSFT', 'NVDA', 'LDO.MI','4816.T']

start_date = datetime(2024, 6, 1)
end_date = datetime(2025, 12, 29)